In [8]:
from pre_processing import aggregate
from display_lib import html_table, ranks
import os
import pandas as pd

In [9]:
# aggregate the data under the chosen metrics
data = aggregate()

# count the number of explicitly missing values
print(f'Missing Values: {data.isna().sum().sum()}')

Error with summaries\Bronze\83e60134-c175-422f-a48d-5659ce2de70a: 'duration'
Error with summaries\GC\439c9822-992c-4d97-bc9b-d5539a02c878: 'match_guid'
Error with summaries\GC\64b54590-ba50-4c20-9157-5521a05a7819: 'match_guid'
Error with summaries\GC\7b4e0791-4115-4dca-a361-eb62c7bc5c39: 'match_guid'
Error with summaries\GC\cd6ec699-9867-428a-93f0-376a5919bf0b: 'match_guid'
Missing Values: 0


In [10]:
# find the number of downloaded summaries for each rank
tot = dict((r.split('\\')[-1], len(f)) for r, _, f in os.walk('summaries'))
dup, dur, rem = {}, {}, {}

for rank, df in data['duration'][::2].groupby('rank'):
  # determine the number of duplicated matches
  dup[rank] = tot[rank] - len(df)
  # determine the number of matches less than half the length of a full match
  dur[rank] = (df < 150).sum()
  rem[rank] = len(df) - dur[rank]

# drop matches which were less than half the length of a full match
data['duration'].where(data['duration'] > 150, pd.NA, inplace=True)
data.dropna(inplace=True)

# display a table with the calculated values
html_table(ranks, [
  ('Total Matches', tot),
  ('Duplicates', dup),
  ('Short Matches', dur),
  ('Remaining Matches', rem)
])

<table style="margin-left:auto;margin-right:auto;"><tr><td></td><td style="text-align:center"><b>Bronze</b></td><td style="text-align:center"><b>Silver</b></td><td style="text-align:center"><b>Gold</b></td><td style="text-align:center"><b>Plat</b></td><td style="text-align:center"><b>Diamond</b></td><td style="text-align:center"><b>Champ</b></td><td style="text-align:center"><b>GC</b></td><td style="text-align:center"><b>SSL</b></td></tr><tr><td style="text-align:center"><b>Total Matches</b></td><td style="text-align:center">180</td><td style="text-align:center">933</td><td style="text-align:center">2877</td><td style="text-align:center">3184</td><td style="text-align:center">3185</td><td style="text-align:center">3185</td><td style="text-align:center">3185</td><td style="text-align:center">3183</td></tr><tr><td style="text-align:center"><b>Duplicates</b></td><td style="text-align:center">7</td><td style="text-align:center">0</td><td style="text-align:center">0</td><td style="text-align:center">9</td><td style="text-align:center">35</td><td style="text-align:center">82</td><td style="text-align:center">130</td><td style="text-align:center">174</td></tr><tr><td style="text-align:center"><b>Short Matches</b></td><td style="text-align:center">106</td><td style="text-align:center">215</td><td style="text-align:center">470</td><td style="text-align:center">361</td><td style="text-align:center">268</td><td style="text-align:center">204</td><td style="text-align:center">145</td><td style="text-align:center">154</td></tr><tr><td style="text-align:center"><b>Remaining Matches</b></td><td style="text-align:center">67</td><td style="text-align:center">718</td><td style="text-align:center">2407</td><td style="text-align:center">2814</td><td style="text-align:center">2882</td><td style="text-align:center">2899</td><td style="text-align:center">2910</td><td style="text-align:center">2855</td></tr></table>

In [11]:
# save the updated data table
data.to_pickle('summaries\\data_frame')